In [1]:
# Imports
import math
import random

In [2]:
def generate_seed_points(seq_points, n_clusters):
  s_x = [x for (x, y) in seq_points]
  s_y = [y for (x, y) in seq_points]
  n_p = len(seq_points)
  x_max = max(s_x)
  x_min = min(s_x)
  y_max = max(s_y)
  y_min = min(s_y)
  size_x = (x_max - x_min)/n_clusters
  size_y = (y_max - y_min)/n_clusters
  n_macro = n_clusters * n_clusters
  delta = average_density(seq_points, n_macro)
  print("x_max", x_max, "x_min", x_min)
  print("y_max", y_max, "y_min", y_min)
  print("size_x", size_x, "size_y", size_y)
  print("number of macroblocks", n_macro, "Average Density", delta)
  s_higher = []

  for i in range(n_clusters):
    x_low = x_min + (i * size_x)
    x_high = x_low + size_x
    x_mid = (x_high + x_low) / 2
    for j in range(n_clusters):
      y_low = y_min + (j * size_y)
      y_high = y_low + size_y
      y_mid = (y_low + y_high)/2
      n_p = points_in_macroblocks(seq_points, x_low, y_low, x_high, y_high)
      print("points in macroblock for", i, j, "is", n_p)
      if n_p > delta:
        s_higher.append((x_mid, y_mid))
    
  
  s_seed = []
  for i in range(n_clusters):
    next_seed = randomly_select(s_higher)
    s_seed.append(next_seed)
    s_higher.remove(next_seed)
  
  radius = min(size_x, size_y)/n_clusters

  print("s_seed", s_seed, "radius", radius)
  for i in range(n_clusters):
    (x_i, y_i) = s_seed[i]
    for j in range(n_clusters):
      if not i == j:
        (x_j, y_j) = s_seed[j]
        d_i_j = math.sqrt(math.pow(x_i - x_j, 2) + math.pow(y_i - y_j, 2))
        print("distance", d_i_j)
        if d_i_j > 2*radius:
          radius = d_i_j/2

  print("radius", radius)
  return (s_seed, radius)


In [3]:
def average_density(seq_points, n_macro):
  return len(seq_points)/n_macro

In [4]:
def randomly_select(s_higher):
  rand_index = random.randint(0, len(s_higher)-1)
  # print("s_higher", s_higher, "rand_index", rand_index)
  return s_higher[rand_index]

In [5]:
def points_in_macroblocks(seq_points, x_low, y_low, x_high, y_high):
  # print("x_low", x_low, "y_low", y_low, "x_high", x_high, "y_high", y_high)
  n_macro = 0
  for (x,y) in seq_points:
    # print("x", x, "y", y)
    if (x >= x_low) and (x < x_high) and (y >= y_low) and (y < y_high):      
      n_macro += 1
  # print("n_macro", n_macro)
  return n_macro


In [6]:
def k_means_clustering(seq_points, n_clusters, max_iter, epi):
  n_p = len(seq_points)
  (s_seed, radius) = generate_seed_points(seq_points, n_clusters)
  print("s_seed", s_seed, "radius", radius)
  count = 1
  stabilized = False

  while((count <= max_iter) and not stabilized):
    print("In Iter", count, "seed points/centroids are", s_seed, "radius is", radius)
    clusters = [[]]
    outliers = seq_points.copy()
    s_out = seq_points
    i = 1
    centroids_new = []
    for i in range(n_clusters):
      # print("i val", i)
      (cx, cy) = s_seed[i]
      for j in range(n_p):
        (x, y) = seq_points[j]
        d_i_j = math.sqrt(math.pow(cx - x, 2) + math.pow(cy - y, 2))
        print(f"distance of ({x},{y}) from centorid ({cx}, {cy}) is {d_i_j}")
        if d_i_j < radius:
          if i >= len(clusters):
            clusters.insert(1, [])
          clusters[i].append((x, y))
          outliers.remove((x, y))
          print("points in cluster", i, "is", clusters[i])
      
      cx_i_new = sum([x for (x, y) in clusters[i]])/len(clusters[i])
      cy_i_new = sum([y for (x, y) in clusters[i]])/len(clusters[i])
      centroids_new.append((cx_i_new, cy_i_new))

      print("cluster", i, "new centroid", (cx_i_new, cy_i_new))

    print("current outliers", outliers)
    stabilized = True

    for i in range(len(clusters)):
      (cx, cy) = s_seed[i]
      (cx_new, cy_new) = centroids_new[i]
      centroid_shift = math.sqrt(math.pow(cx - cx_new, 2) + math.pow(cy - cy_new, 2))
      print("Centroid shift", centroid_shift)
      if centroid_shift > epi:
        stabilized = False
        s_seed[i] = centroids_new[i]

    print()
    count += 1
  
  print("Final clusters")
  for i in range(len(clusters)):
    display(clusters[i])
  print("Final outliers")
  display(outliers)


In [7]:
seq_points = [(1, 2), (2, 3), (2,2), (5, 6), (6, 7), (6, 8), (7, 11), (1, 1)]
n_clusters = 2
max_iter = 10
epi = 1
k_means_clustering(seq_points, n_clusters, max_iter, epi)

x_max 7 x_min 1
y_max 11 y_min 1
size_x 3.0 size_y 5.0
number of macroblocks 4 Average Density 2.0
points in macroblock for 0 0 is 4
points in macroblock for 0 1 is 0
points in macroblock for 1 0 is 0
points in macroblock for 1 1 is 3
s_seed [(2.5, 3.5), (5.5, 8.5)] radius 1.5
distance 5.830951894845301
distance 5.830951894845301
radius 1.5
s_seed [(2.5, 3.5), (5.5, 8.5)] radius 1.5
In Iter 1 seed points/centroids are [(2.5, 3.5), (5.5, 8.5)] radius is 1.5
distance of (1,2) from centorid (2.5, 3.5) is 2.1213203435596424
distance of (2,3) from centorid (2.5, 3.5) is 0.7071067811865476
points in cluster 0 is [(2, 3)]
distance of (2,2) from centorid (2.5, 3.5) is 1.5811388300841898
distance of (5,6) from centorid (2.5, 3.5) is 3.5355339059327378
distance of (6,7) from centorid (2.5, 3.5) is 4.949747468305833
distance of (6,8) from centorid (2.5, 3.5) is 5.70087712549569
distance of (7,11) from centorid (2.5, 3.5) is 8.74642784226795
distance of (1,1) from centorid (2.5, 3.5) is 2.91547594

[(2, 3)]

[(6, 8)]

Final outliers


[(1, 2), (2, 2), (5, 6), (6, 7), (7, 11), (1, 1)]